In [3]:
import os
from datetime import date  # For working with dates

import pandas as pd  # For working with DataFrames
from dotenv import load_dotenv
from sqlalchemy import create_engine  # For crea

##################### LOADING IMPORTANT DATA ######################
# Load environment variables from the .env file
env_file_path = r'D:/Projects/.env'
load_dotenv(env_file_path)
# Giving output file name
output_file_path = 'MONTHLY.xlsx'
# Load data from different sheets in 'promotion.xlsx' into DataFrames
promotion_path = r'D:\Projects\promotion.xlsx'
region_df = pd.read_excel(promotion_path, sheet_name='Region')
aksiya_df = pd.read_excel(promotion_path, sheet_name='Aksiya')
paket_df = pd.read_excel(promotion_path, sheet_name='Paket')
types_df = pd.read_excel(promotion_path, sheet_name='TYPES')
##################### ACCESS ENV VARIABLES ######################
db_server = os.getenv("DB_SERVER")
db_database = os.getenv("DB_DATABASE_SERGELI")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_port = os.getenv("DB_PORT")
db_driver_name = os.getenv("DB_DRIVER_NAME")

##################### PROCEDURE NAME ######################

start_date = date(2024, 3, 1).strftime('%Y%m%d')
end_date = date(2024, 3, 31).strftime('%Y%m%d')

##################### CONNECTION STRING AND SQL QUERY ######################
# Construct the connection string
conn_str = f"mssql+pyodbc://{db_user}:{db_password}@{db_server}:{db_port}/{db_database}?driver={db_driver_name}"
engine = create_engine(conn_str)


# SQL query with placeholders for parameters
sql_query = """
DECLARE @DateBegin DATE = CAST(? AS DATE);
DECLARE @DateEnd DATE = CAST(? AS DATE);
DECLARE @GoodName NVARCHAR(255) = ?;
DECLARE @ClientManager NVARCHAR(255) = ?;

SELECT D.Name                    AS DocKind,
       i.Number                  AS InvoiceNumber,
       G.Name                    AS GoodName,
       M.Name                    AS Producer,
       C.Inn                     as Inn,
       C.FindName                AS ClientName,
       PIM.Name                  AS InvoiceManager,
       PCM.Name                  AS ClientManager,
       I.PaymentTerm,
       il.pBasePrice             as BasePrice,
       il.Price                  as SellingPrice,
       il.Kolich                 AS Quantity,
       I.DataEntered,
       il.pBasePrice * il.Kolich AS BaseAmount,
       il.pSumma                 AS TotalAmount
FROM INVOICELN il
         JOIN INVOICE i ON il.InvoiceId = i.InvoiceId
         JOIN PERSONAL PIM ON i.PersonalId = PIM.PersonalId
         JOIN DOCKIND D ON i.DocKindId = D.DocKindId
         JOIN CLIENT C ON i.ClientId = C.ClientId
         JOIN INCOMELN incl ON il.IncomeLnId = incl.IncomeLnId
         JOIN Good G ON incl.GoodId = G.GoodId
         JOIN Producer M ON M.ProducerId = G.ProducerId
         JOIN PERSONAL PCM ON C.PersonalId = PCM.PersonalId
WHERE i.DataEntered >= @DateBegin
  AND i.DataEntered <= @DateEnd
  AND (D.name = 'Финансовая скидка' OR D.name = 'Оптовая реализация')
  AND (ISNULL(@GoodName, '') = '' OR G.Name = @GoodName)
  AND (ISNULL(@ClientManager, '') = '' OR PCM.Name = @ClientManager)

ORDER BY i.DataEntered DESC
"""

# Execute the query using parameters
df = pd.read_sql_query(sql_query, engine, params=(start_date, end_date,'','(Джиз) Кушбаков Зокир'))


In [4]:
df

,DocKind,InvoiceNumber,GoodName,Producer,Inn,ClientName,InvoiceManager,ClientManager,PaymentTerm,BasePrice,SellingPrice,Quantity,DataEntered,BaseAmount,TotalAmount
0,Оптовая реализация,40828,Нимид 100мг №30(саше),Kusum Healthcare,307012984,Ёрбекфарм-ш мчж,(Джиз) Кушбаков Зокир,(Джиз) Кушбаков Зокир,25,65255.31,72461.83,9.0,2024-03-04 19:06:00,587297.79,652156.47
1,Оптовая реализация,40826,Зиомицин 500мг №3,Kusum Healthcare,307012984,Ёрбекфарм-ш мчж,(Джиз) Кушбаков Зокир,(Джиз) Кушбаков Зокир,25,26521.07,29979.31,12.0,2024-03-04 19:04:00,318252.84,359751.72
2,Оптовая реализация,40824,Аскоцин №100,Kusum Healthcare,307012984,Ёрбекфарм-ш мчж,(Джиз) Кушбаков Зокир,(Джиз) Кушбаков Зокир,25,78261.76,89993.20,4.0,2024-03-04 19:03:00,313047.04,359972.80
3,Оптовая реализация,40821,L-цет 5 мг №100,Kusum Healthcare,307012984,Ёрбекфарм-ш мчж,(Джиз) Кушбаков Зокир,(Джиз) Кушбаков Зокир,25,156329.99,175199.02,16.0,2024-03-04 19:00:00,2501279.84,2803184.32
4,Оптовая реализация,40076,Метформин 850мг №30,Гуфик,307685543,АНТИДАРД ФАРМ дж,(Джиз) Кушбаков Зокир,(Джиз) Кушбаков Зокир,25,25000.00,25000.00,2.0,2024-03-04 12:32:00,50000.00,50000.00
5,Оптовая реализация,40076,Лизинокор 5мг №20,"GM Pharmaceuticals Ltd,Грузия",307685543,АНТИДАРД ФАРМ дж,(Джиз) Кушбаков Зокир,(Джиз) Кушбаков Зокир,25,18924.63,21761.44,5.0,2024-03-04 12:32:00,94623.15,108807.20
6,Оптовая реализация,40076,Терафлю при гриппе и простуды №10 Лимон пор. д...,Делфарм Орлеанс,307685543,АНТИДАРД ФАРМ дж,(Джиз) Кушбаков Зокир,(Джиз) Кушбаков Зокир,25,60384.28,69435.89,12.0,2024-03-04 12:32:00,724611.36,833230.68
7,Оптовая реализация,40076,Преднизолон SD 5мг №50 табл.,Sharq Darmon Узбекистан,307685543,АНТИДАРД ФАРМ дж,(Джиз) Кушбаков Зокир,(Джиз) Кушбаков Зокир,25,13000.00,14948.70,4.0,2024-03-04 12:32:00,52000.00,59794.80
8,Оптовая реализация,40076,Верошпирон 25 мг №20,Gedeon Richter,307685543,АНТИДАРД ФАРМ дж,(Джиз) Кушбаков Зокир,(Джиз) Кушбаков Зокир,25,20932.51,23982.93,2.0,2024-03-04 12:32:00,41865.02,47965.86
9,Оптовая реализация,40076,Лизинокор 10мг №20,"GM Pharmaceuticals Ltd,Грузия",307685543,АНТИДАРД ФАРМ дж,(Джиз) Кушбаков Зокир,(Джиз) Кушбаков Зокир,25,28904.03,33236.75,2.0,2024-03-04 12:32:00,57808.06,66473.50
